In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [5]:
for i in range(50,60):
    print(train_data.text.iloc[i])
    print(train_data.target.iloc[i])

Deputies: Man shot before Brighton home set ablaze http://t.co/gWNRhMSO8k
1
Man wife get six years jail for setting ablaze niece
http://t.co/eV1ahOUCZA
1
SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf
0
Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze http://t.co/pcXarbH9An
1
Noches El-Bestia '@Alexis_Sanchez: happy to see my teammates and training hard ?? goodnight gunners.?????? http://t.co/uc4j4jHvGR'
0
#Kurds trampling on Turkmen flag later set it ablaze while others vandalized offices of Turkmen Front in #Diyala http://t.co/4IzFdYC3cg
1
TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE OR TAMBO INTL. CARGO SECTION. http://t.co/8kscqKfKkF
1
Set our hearts ablaze and every city was a gift And every skyline was like a kiss upon the lips @Û_ https://t.co/cYoMPZ1A0Z
0
They sky was ablaze tonight in Los Angeles. I'm expecting IG and FB to be filled with sunset shots if I know my p

Some very basic cleanup work to start.  Mostly just set all URLs and @terms to respective keywords

In [6]:
from string import punctuation
import re
from nltk.corpus import stopwords
import nltk
import emoji
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text)
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

stopWords = set(stopwords.words('english'))

def clean_text(text):
    # change URL/@ tags to keywords
    text = ' '.join(['URL' if 'http://' in i or 'https://' in i else i for i in text.split(' ') ])
    text = ' '.join(['ATUSER' if len(i)>0 and i[0]=='@' else i for i in text.split(' ') ])
    #remove punctuation
    text = ''.join([c for c in text if c not in punctuation])
    #replace newline with space
    text = re.sub('\n',' ',text)
    #remove extra spaces
    text = re.sub(' +', ' ', text)
    #remove stop words
    text = ' '.join([i for i in text.lower().split(' ') if i not in stopWords])
    return text

def cleanTweet(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+','',txt)
    txt = re.sub(r'#','',txt)
    txt = re.sub(r'RT : ','',txt)
    txt = re.sub(r'\n','',txt)
    # to remove emojis
    txt = re.sub(emoji.get_emoji_regexp(), r"", txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+','',txt)
    txt = re.sub(r"https?://\S+|www\.\S+","",txt)
    txt = re.sub(r"<.*?>","",txt)
    return txt  

def remove_punct(text):
    table=str.maketrans('','',punctuation)
    return text.translate(table)

def make_Lower(text):
    return str.lower(text)

#text = [clean_text(i) for i in train_data.text]
train_data.text = train_data.text.apply(clean_text)
#train_data.text = train_data.text.apply(make_Lower)
train_data.text = train_data.text.apply(correct_spellings)
#train_data.text = train_data.text.apply(remove_punct)
text = train_data.text.tolist()

#text = [clean_text(i) for i in train_data.text]
test_data.text = test_data.text.apply(clean_text)
#test_data.text = test_data.text.apply(remove_punct)
#test_data.text = test_data.text.apply(make_Lower)
test_data.text = test_data.text.apply(correct_spellings)
#test_data.text = test_data.text.apply(remove_punct)
test_text = test_data.text.tolist()

In [7]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

all_words = flatten_list([i.split(' ') for i in text]) + flatten_list([i.split(' ') for i in test_text])

In [8]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts,key=counts.get, reverse=True)
vocab_to_int = {word:i for i,word in enumerate(vocab,1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
tweet_ints = []
for tweet in text:
    tweet_ints.append([vocab_to_int[word] for word in tweet.split(' ')])
    
tweet_ints_test = []
for tweet in test_text:
    tweet_ints_test.append([vocab_to_int[word] for word in tweet.split(' ') if word in vocab_to_int.keys()])
    

tweet_ints_test = [i if len(i)>0 else [0] for i in tweet_ints_test]
tweet_ints = [i if len(i)>0 else [0] for i in tweet_ints]

In [9]:
#how big is vocab?
len(vocab)

19578

In [10]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

#generate the embedding matrix
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens+1, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i+1] = nlp(word).vector

In [11]:
spacy_embedding_dim = embedding_matrix.shape

In [12]:
# outlier review stats
tweet_lens = Counter([len(x) for x in tweet_ints])
print("Zero-length reviews: {}".format(tweet_lens[0]))
print("Maximum length: {}".format(max(tweet_lens)))

Zero-length reviews: 0
Maximum length: 25


In [13]:
# pad tweets to equal legnths
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

seq_length = 25

features = pad_features(tweet_ints, seq_length=seq_length)
sub_features = pad_features(tweet_ints_test, seq_length=seq_length)

In [14]:
split_frac = 0.8

## split data into training, validation data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, test_x = features[:split_idx], features[split_idx:]
train_y, test_y = train_data.target.tolist()[:split_idx], train_data.target.tolist()[split_idx:]

print(train_x[0])
print(train_y[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0 5529  698  150   67 1912 3520   15]
1


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))
sub_data = TensorDataset(torch.from_numpy(np.array(sub_features)), torch.from_numpy(np.zeros(len(sub_features))))

# dataloaders
batch_size = 32

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)
sub_loader = DataLoader(sub_data, shuffle=False, batch_size=1,drop_last=True)

In [16]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([32, 25])
Sample input: 
 tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   773,   213,   185, 12625, 12626,  2493,
           383,   320,  5157, 12627,  1504],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     2,     2,    56,  2180,     3,
            25,    41,   553,    75,    35],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     2,     2, 13139,  1178,  4139,
          2013, 13140,  6850,  7204,   211],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  1681,   773,  1401, 13234,  3664,   352,   117, 13235,   187,
             3,  4946,   339,  4665, 13236],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [17]:
train_on_gpu=torch.cuda.is_available()

In [18]:
import torch.nn as nn

class ClassifierRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(ClassifierRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight=nn.Parameter(torch.tensor(embedding_matrix,dtype=torch.float32))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,bidirectional=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
      
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        #batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)

        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [19]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = spacy_embedding_dim[1]
hidden_dim = 256
n_layers = 2

net = ClassifierRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

ClassifierRNN(
  (embedding): Embedding(19579, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [20]:
# loss and optimization functions
lr=0.00001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
PATH = 'mdl'

In [ ]:
# training params

epochs = 10

counter = 0
print_every = 50
clip=5 # gradient clipping
best = 999

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in test_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                if val_loss < best:
                    torch.save(net,PATH)
                    best = val_loss

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
# Get test data loss and accuracy

net = torch.load(PATH)
net.eval()

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    #h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

In [ ]:
# Get test data loss and accuracy
all_preds = []
net = torch.load(PATH)
batch_size =1

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data

for inputs, labels in sub_loader:
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    #h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()).item()  # rounds to the nearest integer
    all_preds.append(pred)



In [ ]:
all_preds

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.shape

In [ ]:
test_data['target'] = all_preds

In [ ]:
test_data.filter(['id','target']).to_csv('sub.csv',index=False)

In [ ]:
test_data